# INFO-4604/5604 HW2: Linear Classification 
## Deadline: Wednesday, October 3, 9:00pm MT

### Solution by: *YOUR NAME* (and list any partners)


## Assignment overview

In this assignment, you will build a classifier that tries to infer whether tweets from [@realDonaldTrump](https://twitter.com/realDonaldTrump) were written by Trump himself or by a staff person.
This is an example of binary classification on a text dataset. 

It is known that Donald Trump uses an Android phone, and it has been observed that some of his tweets come from Android while others come from other devices (most commonly iPhone). It is widely believed that Android tweets are written by Trump himself, while iPhone tweets are written by other staff. For more information, you can read this [blog post by David Robinson](http://varianceexplained.org/r/trump-tweets/), written prior to the election, which finds a number of differences in the style and timing of tweets published under these two devices. (Some tweets are written from other devices, but for simplicity the dataset for this assignment is restricted to these two.)

This is a classification task known as "authorship attribution", which is the task of inferring the author of a document when the authorship is unknown. We will see how accurately this can be done with linear classifiers using word features.

You will need `sklearn` version 0.23 for this assignment.

### What to hand in

You will submit the assignment on Canvas. Submit a single Jupyter notebook named `hw2lastname.ipynb`, where lastname is replaced with your last name.

If you have any output that is not part of your notebook, you may submit that as a separate document, in a single PDF named `hw2lastname.pdf`. For example, this assignment requires you to create plots. You could do it directly with python using [matplotlib](https://matplotlib.org/), but if you wanted to create them using other software, that's acceptable as long as you put all of the figures in a single document and you clearly label them with the corresponding deliverable number.

When writing code in this notebook, you are encouraged to create additional cells in whatever way makes the presentation more organized and easy to follow. You are allowed to import additional Python libraries.

### Submission policies

- **Collaboration:** You are allowed to work with up to 3 people besides yourself. You are still expected to write up your own solution. Each individual must turn in their own submission, and list your collaborators after your name.
- **Late submissions:** We allow each student to use up to 5 late days over the semester. You have late days, not late hours. This means that if your submission is late by any amount of time past the deadline, then this will use up a late day. If it is late by any amount beyond 24 hours past the deadline, then this will use a second late, and so on. Once you have used up all late days, late assignments will be given at most 80% credit after one day and 60% credit after two days.


## Getting started

In this assignment, you will experiment with perceptron and logistic regression in `sklearn`. Much of the code has already been written for you. We will use a class called `SGDClassifier` (which you should read about in the [sklearn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)), which  implements stochastic gradient descent (SGD) for a variety of loss functions, including both perceptron and logistic regression, so this will be a way to easily move between the two classifiers.

The code below will load the datasets. There are two data collections: the "training" data, which contains the tweets that you will use for training the classifiers, and the "testing" data, which are tweets that you will use to measure the classifier accuracy. The test tweets are instances the classifier has never seen before, so they are a good way to see how the classifier will behave on data it hasn't seen before. However, we still know the labels of the test tweets, so we can measure the accuracy.

For this problem, we will use what are called "bag of words" features, which are commonly used when doing classification with text. Each feature is a word, and the value of a feature for a particular tweet is number of times the word appears in the tweet (with value $0$ if the word does not appear in the tweet).

Run the block of code below to load the data. You don't need to do anything yet. Move on to "Problem 1" next.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

df_train = pd.read_csv('http://cmci.colorado.edu/classes/INFO-4604/data/tweets.train.tsv', sep='\t', header=None)

Y_train = df_train.iloc[0:, 0].values
text_train = df_train.iloc[0:, 1].values

vec = CountVectorizer()
X_train = vec.fit_transform(text_train)
feature_names = np.asarray(vec.get_feature_names())

df_test = pd.read_csv('http://cmci.colorado.edu/classes/INFO-4604/data/tweets.test.tsv', sep='\t', header=None)
Y_test = df_test.iloc[0:, 0].values
text_test = df_test.iloc[0:, 1].values

X_test = vec.transform(text_test)


## Problem 1: Understand the data [6 points]

Before doing anything else, take time to understand the code above.

The variables `df_train` and `df_test` are dataframes that store the training (and testing) datasets, which are contained in tab-separated files where the first column is the label and the second column is the text of the tweet.

The [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) class converts the raw text into a bag-of-words into a feature vector representation that `sklearn` can use.

You should print out the values of the variables and write any other code needed to answer the following questions.

#### Deliverable 1.1: How many training instances are in the dataset? How many test instances?

[your answer here]

#### Deliverable 1.2: How many features are in the training data?

[your answer here]

#### Deliverable 1.3: What is the distribution of labels in the training data? That is, what percentage of instances are 'Android' versus 'iPhone'?

[your answer here]

## Problem 2: Perceptron [6 points]

The code below trains an [`SGDClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) using the perceptron loss, then it measures the accuracy of the classifier on the test data, using `sklearn`'s [`accuracy_score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function. 

The `fit` function trains the classifier. The feature weights are stored in the `coef_` variable after training. The `predict` function of the trained `SGDClassifier` outputs the predicted label for a given instance or list of instances.

Additionally, this code displays the features and their weights in sorted order, which you may want to examine to understand what the classifier is learning. In this dataset, the $\textrm{Android}$ class is considered the "negative" class because it comes first in the data.

There are 3 keyword arguments that have been added to the code below. It is important you keep the same values of these arguments whenever you create an `SGDClassifier` instance in this assignment so that you get consistent results. They are:

- `max_iter` is one of the stopping criteria, which is the maximum number of iterations/epochs the algorithm will run for.

- `tol` is the other stopping criterion, which is how small the difference between the current loss and previous loss should be before stopping.

- `random_state` is a seed for pseudorandom number generation. The algorithm uses randomness in the way the training data are sorted, which will affect the solution that is learned, and even the accuracy of that solution.

Wait a minute $-$ in class we learned that the loss function is convex, so the algorithm will find the same minimum regardless of how it is trained. Why is there random variation in the output? The reason is that even though there is only one minimum value of the loss, there may be different weights that result in the same loss, so randomness is a matter of tie-breaking. What's more, while different weights may have the same loss, they could lead to different classification accuracies, because the loss function is not the same as accuracy. (Unless accuracy was your loss function... which is possible, but uncommon because it turns out to be a difficult function to optimize.)

Note that different computers may still give different answers, despite keeping these settings the same, because of how pseudorandom numbers are generated with different operating systems and Python environments. 

To begin, run the code in the cell below without modification.

#### Deliverable 2.1: Based on the training accuracy, do you conclude that the data are linearly separable? Why or why not?

[your answer here]

#### Deliverable 2.2: Which feature most increases the likelihood that the class is 'Android' and which feature most increases the likelihood that the class is 'iPhone'? 

[your answer here]

<br />

One technique for improving the resulting model with perceptron (or stochastic gradient descent learning in general) is to take an average of the weight vectors learned at different iterations of the algorithm, rather than only using the final weights that minimize the loss. That is, calculate $\bar{\mathbf{w}} = \sum_{t=1}^T \mathbf{w}^{(t)}$ where $\mathbf{w}^{(t)}$ is the weight vector at iteration $t$ of the algorithm and $T$ is the number of iterations, and then use $\bar{\mathbf{w}}$ when making classifications on new data.

To use this technique in your classifier, add the keyword argument `average=True` to the `SGDClassifier` function. Try it now.

#### Deliverable 2.3: Compare the initial training/test accuracies to the training/test accuracies after doing averaging. What happens? Why do you think averaging the weights from different iterations has this effect?

[your answer here]


In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score

classifier = SGDClassifier(loss='perceptron', max_iter=1000, tol=1.0e-12, random_state=123, eta0=100)
classifier.fit(X_train, Y_train)

print("Number of SGD iterations: %d" % classifier.n_iter_)
print("Training accuracy: %0.6f" % accuracy_score(Y_train, classifier.predict(X_train)))
print("Testing accuracy: %0.6f" % accuracy_score(Y_test, classifier.predict(X_test)))

print("\nFeature weights:")
args = np.argsort(classifier.coef_[0])
for a in args:
    print(" %s: %0.4f" % (feature_names[a], classifier.coef_[0][a]))

Number of SGD iterations: 38
Training accuracy: 0.997300
Testing accuracy: 0.864865

Feature weights:
 00: -1.5070
 veterans: -1.2056
 wow: -1.2056
 talking: -1.1052
 called: -1.1052
 badly: -1.0047
 into: -1.0047
 stay: -0.9042
 actually: -0.9042
 hillaryclinton: -0.9042
 woman: -0.9042
 talks: -0.9042
 look: -0.9042
 weak: -0.8038
 mails: -0.8038
 illegals: -0.8038
 standing: -0.8038
 allowed: -0.8038
 care: -0.8038
 making: -0.8038
 spent: -0.8038
 donaldtrump: -0.8038
 sources: -0.8038
 wrong: -0.8038
 cruz: -0.7033
 reported: -0.7033
 scared: -0.7033
 frontrunner: -0.7033
 expensive: -0.7033
 four: -0.7033
 game: -0.7033
 old: -0.7033
 oregon: -0.7033
 use: -0.7033
 mr: -0.7033
 order: -0.7033
 texas: -0.7033
 charge: -0.7033
 reviews: -0.7033
 statement: -0.7033
 _username_: -0.7033
 taking: -0.6028
 fake: -0.6028
 presumptive: -0.6028
 turned: -0.6028
 sigh: -0.6028
 read: -0.6028
 teleprompter: -0.6028
 tim: -0.6028
 doesn: -0.6028
 isn: -0.6028
 many: -0.6028
 senator: -0.6028

 daily: -0.1005
 blamed: -0.1005
 admire: -0.1005
 patriots: -0.1005
 staying: -0.1005
 curse: -0.1005
 yourself: -0.1005
 minnesota: -0.1005
 misleading: -0.1005
 lousy: -0.1005
 disloyalty: -0.1005
 leadership: -0.1005
 top: -0.1005
 limbaugh: -0.1005
 necessary: -0.1005
 proudly: -0.1005
 flooding: -0.1005
 french: -0.1005
 rationally: -0.1005
 readership: -0.1005
 departed: -0.1005
 tens: -0.1005
 prosperity: -0.1005
 causing: -0.1005
 his: -0.1005
 priest: -0.1005
 kasich: -0.1005
 doj: -0.1005
 beloved: -0.1005
 13: -0.1005
 sickness: -0.1005
 weakening: -0.1005
 packed: -0.1005
 placed: -0.1005
 existent: -0.1005
 watchdog: -0.1005
 crafted: -0.1005
 unnamed: -0.1005
 therefore: -0.1005
 sleep: -0.1005
 forgot: -0.1005
 destructive: -0.1005
 combined: -0.1005
 average: -0.1005
 losing: -0.1005
 track: -0.1005
 skills: -0.1005
 russian: -0.1005
 connection: -0.1005
 seek: -0.1005
 reading: -0.1005
 telepromter: -0.1005
 appointed: -0.1005
 more: -0.1005
 justices: -0.1005
 here: 

 100k: 0.0000
 00am: 0.0000
 youth: 0.0000
 youtube: 0.0000
 yr: 0.0000
 yt: 0.0000
 zealand: 0.0000
 youngstown: 0.0000
 wilmington: 0.0000
 written: 0.0000
 wounds: 0.0000
 windham: 0.0000
 winners: 0.0000
 147: 0.0000
 143: 0.0000
 wisdom: 0.0000
 139: 0.0000
 10am: 0.0000
 1314: 0.0000
 12m: 0.0000
 women4trump: 0.0000
 wondered: 0.0000
 11a: 0.0000
 worker: 0.0000
 worry: 0.0000
 12pm: 0.0000
 undermines: 0.0000
 walker: 0.0000
 wake: 0.0000
 unrest: 0.0000
 unsubstantiated: 0.0000
 300: 0.0000
 untrusting: 0.0000
 unverifiable: 0.0000
 unverified: 0.0000
 unrelenting: 0.0000
 unwatchable: 0.0000
 updated: 0.0000
 2m: 0.0000
 uranium: 0.0000
 urged: 0.0000
 urging: 0.0000
 290: 0.0000
 2pm: 0.0000
 28: 0.0000
 unquestionably: 0.0000
 unprofessional: 0.0000
 undoing: 0.0000
 unenthusiastic: 0.0000
 unfortunately: 0.0000
 325: 0.0000
 unheard: 0.0000
 unify: 0.0000
 unqualified: 0.0000
 unions: 0.0000
 30pme: 0.0000
 30p: 0.0000
 unleash: 0.0000
 30am: 0.0000
 306: 0.0000
 unprecede

 hampshire: 0.1005
 nights: 0.1005
 disney: 0.1005
 forge: 0.1005
 frozen: 0.1005
 puppet: 0.1005
 corporate: 0.1005
 pushes: 0.1005
 refusal: 0.1005
 2004: 0.1005
 ambushed: 0.1005
 truthful: 0.1005
 550: 0.1005
 lodge: 0.1005
 passing: 0.1005
 fraternal: 0.1005
 vibrant: 0.1005
 train: 0.1005
 area: 0.1005
 active: 0.1005
 retired: 0.1005
 kenansville: 0.1005
 gravy: 0.1005
 agitators: 0.1005
 342: 0.1005
 oversaw: 0.1005
 00pm: 0.1005
 cooper: 0.1005
 operative: 0.1005
 anderson: 0.1005
 jackson: 0.1005
 winning: 0.1005
 ministries: 0.1005
 bishop: 0.1005
 wayne: 0.1005
 11pm: 0.1005
 international: 0.1005
 might: 0.1005
 trust: 0.1005
 successful: 0.1005
 blessings: 0.1005
 easier: 0.1005
 prosperous: 0.1005
 happynewyear: 0.1005
 extended: 0.1005
 absolutely: 0.1005
 newtown: 0.1005
 judgement: 0.1005
 bowl: 0.1005
 airs: 0.1005
 herman: 0.1005
 purpose: 0.1005
 report: 0.1005
 agrees: 0.1005
 mongering: 0.1005
 americanunity: 0.1005
 division: 0.1005
 fear: 0.1005
 era: 0.1005
 r

## Problem 3: Logistic regression [15 points]

For this problem, create a new `SGDClassifier`, this time setting the `loss` argument to `'log'`, which will train a logistic regression classifier. Set `average=False` for the remaining problems.

Once you have trained the classifier, you can use the `predict` function to get the classifications, as with perceptron. Additionally, logistic regression provides probabilities for the predictions. You can get the probabilities by calling the `predict_proba` function. This will give a list of two numbers; the first is the probability that the class is $\textrm{Android}$ and the second is the probability that the class is $\textrm{iPhone}$.


For the first task, add the keyword argument `alpha` to the `SGDClassifier` function. This is the regularization strength, called $\lambda$ in lecture. If you don't specify `alpha`, it defaults to $0.0001$. Experiment with other values and see how this affects the outcome.

#### Deliverable 3.1: Calculate the training and testing accuracy when `alpha` is one of $[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]$. Create a plot where the x-axis is `alpha` and the y-axis is accuracy, with two lines (one for training and one for testing). You can borrow the code from HW1 for generating plots in Python. Use [a log scale for the x-axis](https://matplotlib.org/examples/pylab_examples/log_demo.html) so that the `alpha` values are spaced evenly.

[your solution should either be plotted below, or included in a separate PDF]

#### Deliverable 3.2: Examine the classifier probabilities using the `predict_proba` function when training with different values of `alpha`. What do you observe? How does `alpha` affect the prediction probabilities, and why do you think this happens?

[your answer here]

<br />

Now remove the `alpha` argument so that it goes back to the default value. We'll now look at the effect of the learning rate. By default, `sklearn` uses an "optimal" learning rate based on some heuristics that work well for many problems. However, it can be good to see how the learning rate can affect the algorithm.

For this task, add the keyword argument `learning_rate` to the `SGDClassifier` function and set the value to `invscaling`. This defines the learning rate at iteration $t$ as: $\eta_t = \frac{\eta_0}{t^a}$, where $\eta_0$ and $a$ are both arguments you have to define in the `SGDClassifier` function, called `eta0` and `power_t`, respectively. Experiment with different values of `eta0` and `power_t` and see how they affect the number of iterations it takes the algorithm to converge. You will often find that it will not finish within the maximum of $1000$ iterations.

#### Deliverable 3.3: Fill in the table below with the number of iterations for values of `eta0` in $[10.0, 100.0, 1000.0, 10000.0]$ and values of `power_t` in $[0.5, 1.0, 2.0]$. You may find it easier to write python code that can output the markdown for the table, but if you do that place the output here. If it does not converge within the maximum number of iterations (set to $1000$ by `max_iter`), record $1000$ as the number of iterations. You will need to read the documentation for this class to learn how to recover the actual number of iterations before reaching the stopping criterion.

| `eta0`   | `power_t` | # Iterations |
|-----------|-----------|--------------|
| $10.0$    | $0.5$     |              |
| $10.0$    | $1.0$     |              |
| $10.0$    | $2.0$     |              |
| $100.0$   | $0.5$     |              |
| $100.0$   | $1.0$     |              |
| $100.0$   | $2.0$     |              |
| $1000.0$  | $0.5$     |              |
| $1000.0$  | $1.0$     |              |
| $1000.0$  | $2.0$     |              |
| $10000.0$ | $0.5$     |              |
| $10000.0$ | $1.0$     |              |
| $10000.0$ | $2.0$     |              |

#### Deliverable 3.4: Describe how `eta0` and `power_t` affect the learning rate based on the formula (e.g., if you increase `power_t`, what will this do to the learning rate?), and connect this to what you observe in the table above.

[your answer here]
   
<br />

Now remove the `learning_rate`, `eta0`, and `power_t` arguments so that the learning rate returns to the default setting. For this final task, we will experiment with how high the probabiity must be before an instance is classified as positive.

The code below includes a function called `threshold` which takes as input the classification probabilities of the data (called `probs`, which is given by the function `predict_proba`) and a threshold (called `tau`, a scalar that should be a value between $0$ and $1$). It will classify each instance as $\textrm{Android}$ if the probability of being $\textrm{Android}$ is greater than `tau`, otherwise it will classify the instance as $\textrm{iPhone}$. Note that if you set `tau` to $0.5$, the `threshold` function should give you exactly the same output as the classifier `predict` function.

You should find that increasing the threshold causes the accuracy to drop. This makes sense, because you are classifying some things as $\textrm{iPhone}$ even though it's more probable that they are $\textrm{Android}$. So why do this? Suppose you care more about accurately identifying the $\textrm{Android}$ tweets and you don't care as much about `iPhone` tweets. You want to be confident that when you classify a tweet as $\textrm{Android}$ that it really is $\textrm{Android}$.

There is a metric called _precision_ which measures something like accuracy but for one specific class. Whereas accuracy is the percentage of tweets that were correctly classified, the precision of $\textrm{Android}$ would be the percentage of tweets classified as $\textrm{Android}$ that were correctly classified. (In other words, the number of tweets classified as $\textrm{Android}$ whose correct label was $\textrm{Android}$, divided by the number of tweets classified as $\textrm{Android}$.)

You can use the [`precision_score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score) function from `sklearn` to calculate the precision. It works just like the `accuracy_score` function, except you have to add an additional keyword argument, `pos_label='Android'`, which tells it that $\textrm{Android}$ is the class you want to calculate the precision of.

#### Deliverable 3.5: Calculate the testing precision when the value of `tau` for thresholding is one of $[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]$. Create a plot where the x-axis is `tau` and the y-axis is precision.

[your solution should either be plotted below, or included in a separate PDF]

#### Deliverable 3.6: Describe what you observe with thresholding (e.g., what happens to precision as the threshold increases?), and explain why you think this happens.

[your answer here]

In [ ]:
# use this function for deliverable 3.5
def threshold(probs, tau):
    return np.where(probs[:,0] > tau, 'Android', 'iPhone')

# your logistic regression code here

classifier = SGDClassifier(loss='log', max_iter=1000, tol=1.0e-12, random_state=123)
classifier.fit(X_train, Y_train)

## Problem 4: Sparse learning [5604: 5 points; 4604: +3 EC points]

Add the `penalty` argument to `SGDClassifier` and set the value to `'l1'`, which tells the algorithm to use L1 regularization instead of the default L2. Recall from lecture that L1 regularization encourages weights to stay at exactly $0$, resulting in a more "sparse" model than L2. You should see this effect if you examine the values of `classifier.coef_`.

#### Deliverable 4.1: Write a function to calculate the number of features whose weights are nonzero when using L1 regularization. Calculate the number of nonzero feature weights when `alpha` is one of $[0.00001, 0.0001, 0.001, 0.01, 0.1]$. Create a plot where the x-axis is `alpha` and the y-axis is the number of nonzero weights, using a log scale for the x-axis.

[your solution should either be plotted below, or included in a separate PDF]

In [ ]:
# your code here
